In [1]:
import pandas
import json
import re
import config

In [2]:
comments = pandas.read_csv('~/Документы/Данные исследования/1fold/Комментарии.csv', sep=';')
posts = pandas.read_csv('~/Документы/Данные исследования/1fold/Посты.csv', sep=';')
stats = pandas.read_csv('~/Документы/Данные исследования/1fold/Анализ Вовлечённости.csv', sep=';')
active = pandas.read_csv('~/Документы/Данные исследования/1fold/Активность.csv', sep=';')
diag1 = pandas.read_csv('~/Документы/Данные исследования/1fold/Диагноcтика.csv')
diag2 = pandas.read_csv('~/Документы/Данные исследования/1fold/ДиагностикаHEOFFI.csv')

In [3]:
def template():
    usrs = {}
    for usr in stats['ID']:
        usrs[usr] = {}
    for usr in usrs:
        usrs[usr]['Посты'] = []
        usrs[usr]['Комментарии'] = []
        usrs[usr]['Диагностика'] = {}
        usrs[usr]['Дополнительно'] = {}
    return usrs


def parse_comms(usrs):
    for i, lnk in enumerate(comments['Ссылка']):
        res = re.search(r'\d+_', lnk)
        com = {'Текст' : comments['Текст'][i], 
               'Лайков' : str(comments['Лайков'][i]),
               'Дата' : str(comments['Дата'][i])}
        usr = res.group(0)[:-1]
        usrs[int(usr)]['Комментарии'].append(com)
    
    
def parse_posts(usrs):
    for i, lnk in enumerate(posts['Ссылка']):
        res = re.search(r'\d+_', lnk)
        pst = {'Текст' : posts['Текст'][i], 
               'Лайков' : str(posts['Лайков'][i]), 
               'Комментариев' : str(posts['Комментариев'][i]), 
               'Репостов' : str(posts['Репостов'][i]), 
               'Просмотров' : str(posts['Просмотров'][i]),
               'Дата' : str(posts['Дата'][i])}
        usr = res.group(0)[:-1]
        usrs[int(usr)]['Посты'].append(pst)
      
    
def parse_stats(usrs):
    for i, usr in enumerate(stats['ID']):
        usr = int(usr)
        usrs[usr]['Активность'] = {'Друзей' : str(stats['Друзей'][i]),
                                   'Подписчиков' : str(stats['Подписчиков'][i]), 
                                   'Лайков на странице' : str(stats['Лайков'][i]),
                                   'Репостов со страницы' : str(stats['Репостов'][i]), 
                                   'Комментариев на странице' : str(stats['Комментариев'][i]),
                                   'Постов оставленных другими пользователями на странице' : str(stats['Постов'][i]),
                                   'Постов на странице': str(len(usrs[usr]['Посты']))}
    
    
def parse_active(usrs):
    used = []
    for i, usr in enumerate(active['ID']):
        if usr in usrs.keys():
            dict_ = {'Постов оставленных пользователем' : str(active['Постов'][i]),
                     'Комментариев оставленных пользователем' : str(active['Комментариев'][i]),
                     'Лайков поставленных пользователем' : str(active['Лайков'][i]), 
                     'Репостов сделанных пользователем' : str(active['Репостов'][i]), 
                     'Опросов созданных пользователем' : str(active['Опросов'][i]),
                     'Обсуждений созданных пользователем' : str(active['Обсуждений'][i]),
                     'Всего активность' : str(active['Всего'][i])}
            used.append(usr)
            usrs[usr]['Активность'].update(dict_)
            
    for usr in usrs.keys():
        if not (usr in used):
            dict_ = {'Постов оставленных пользователем' : '0',
                     'Комментариев оставленных пользователем' : '0',
                     'Лайков поставленных пользователем' : '0',
                     'Репостов сделанных пользователем' : '0', 
                     'Опросов созданных пользователем' : '0',
                     'Обсуждений созданных пользователем' : '0',
                     'Всего активность' : '0'}
            usrs[usr]['Активность'].update(dict_)
        
            
def parse_diag1(usrs):
    for i, usr in enumerate(diag1['1']):
        sum = 0
        for j in range(7, 27):
            sum += int(diag1[str(j)][i][-1]) - 1
        usrs[usr]['Диагностика']['Депрессия'] = str(sum)

        
def parse_diag2(usrs):
    for i, usr in enumerate(diag2['1']):
        try:
            usrs[usr]['Диагностика']['Нейротизм'] = str(diag2['3'][i])
            usrs[usr]['Диагностика']['Экстраверсия'] = str(diag2['4'][i])
            usrs[usr]['Диагностика']['Открытость опыту'] = str(diag2['5'][i])
            usrs[usr]['Диагностика']['Готовность к согласию'] = str(diag2['6'][i])
            usrs[usr]['Диагностика']['Сознательность'] = str(diag2['7'][i])
        except KeyError:
            pass
          
        
# разных комментаторов
# среднее количество слов в посте
# среднее количество предложений в посте 

def parse_dop(usrs):
    
    # среднее количество предложений в посте
    # среднее количество слов в посте
    
    # уникальные комментаторы
    spec_dict = {}
    for u in usrs:
        spec_dict[u] = {}
        spec_dict[u]['Уникальных комментаторов'] = 0
        spec_dict[u]['Комментаторы'] = []
    for i, lnk in enumerate(comments['Ссылка']):
        res = re.search(r'\d+_', lnk)
        u = int(res.group(0)[:-1])
        autor = comments['Автор'][i]
        if autor in spec_dict[u]['Комментаторы']:
            pass
        else:
            spec_dict[u]['Комментаторы'].append(autor)
            spec_dict[u]['Уникальных комментаторов'] += 1
    for u in usrs:
        usrs[u]['Дополнительно']['Уникальных комментаторов'] = str(spec_dict[u]['Уникальных комментаторов'])
    
    #плотность активности
    
    for usr in usrs:
        d = usrs[usr]['Активность']
        try:
            usrs[usr]['Дополнительно'][
                'Плотность активности друзей'
                                      ] = str((int(d['Лайков на странице']) + int(d['Комментариев на странице'])) / int((d['Друзей'])) + int(d['Подписчиков']))
        except ZeroDivisionError:
            usrs[usr]['Дополнительно']['Плотность активности друзей'] = '0'
    
    
def start():
    usrs = template()
    parse_comms(usrs)
    parse_posts(usrs)
    parse_stats(usrs)
    parse_active(usrs)
    parse_diag1(usrs)
    parse_diag2(usrs)
    parse_dop(usrs)
    return usrs

In [4]:
with open(config.PATH_USRS, 'w') as file:
    json.dump(start(), file,  ensure_ascii=False)